In [ ]:
!pip install pydub
!pip install librosa

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pydub
from scipy import signal
import librosa
import librosa.display

def get_samples(sound):
    channel_count = sound.channels
    if channel_count == 1:
        samples = np.array(sound.get_array_of_samples())
    elif channel_count == 2:
        sound_mono = sound.split_to_mono()
        samples = np.array(sound_mono[0].get_array_of_samples())
    else:
        raise Exception("More than 2 channels!")
    return samples

def plot_stft(t, f, dB, anchors=None):

    params = {'legend.fontsize': 'x-large','figure.figsize': (25, 6),
         'axes.labelsize': 'x-large','axes.titlesize':'x-large',
         'xtick.labelsize':'x-large','ytick.labelsize':'x-large'}
    plt.rcParams.update(params)

    fig = plt.figure()
    plt.pcolormesh(t, f, dB, cmap='YlOrRd', shading='auto')
    fig.suptitle('Short-time Fourier transform (window duration 50ms, hopsize 10 ms)')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (kHz)')
    plt.colorbar(format='%+02.0f dB')
    plt.ylim([0, 5])
    if anchors is not None:
        anchors_t, anchors_f = map(list, zip(*anchors))
        plt.scatter(anchors_t, anchors_f, marker='x', color='black')
    plt.show()
    
def generate_anchors(t, f, dB):
    delta_t = 0.1
    f_num_bands = 25

    t_rate = len(t) / (t[-1] - t[0])
    delta_i = int(delta_t * t_rate)
    if delta_i == 0:
        delta_i = 1
    delta_j = int(len(f) / f_num_bands)

    anchors_t = []
    anchors_f = []
    anchors = []

    for i in range(0, len(t) - delta_i, delta_i):
        for j in range(0, len(f) - delta_j, delta_j):
            tmp = dB[j:j+delta_j, i:i+delta_i]
            ind = np.unravel_index(np.argmax(tmp, axis=None), tmp.shape)
            anchors_t.append(t[i + ind[1]])
            anchors_f.append(f[j + ind[0]])
            anchors.append((t[i + ind[1]], f[j + ind[0]]))
    
    return anchors

def generate_fingerprint(anchors):
    F = []
    t_offset = 0.1
    t_window = 0.5
    f_factors = (2**-0.5, 2**0.5)

    anchors.sort(key=lambda x: x[0])
    for t1, f1 in anchors:
        t_min = t1 + t_offset
        t_max = t1 + t_offset + t_window
        f_min = f1 * f_factors[0]
        f_max = f1 * f_factors[1]
        for t2, f2 in anchors:
            if t2 < t_min or t2 > t_max or f2 < f_min or f2 > f_max:
                continue
            h = (f1, f2, t2 - t1)
            F.append((t1, h))
    
    return F

def generate_matches(F1, F2):
    matches = []
    h_range = range(len(F1[0][1]))
    for t1, h1 in F1:
        for t2, h2 in F2:
            match = True
            for i in h_range:
                if h1[i] != h2[i]:
                    match = False
                    break
            if match:
                matches.append((t1, t2))
    return matches

def generate_num_of_matches(F1, F2):
    num_of_matches = 0
    h_range = range(len(F1[0][1]))
    for t1, h1 in F1:
        for t2, h2 in F2:
            match = True
            for i in h_range:
                if h1[i] != h2[i]:
                    match = False
                    break
            if match:
                num_of_matches = num_of_matches + 1
    return num_of_matches

def plot_matches(matches, self_compare=False):
    params = {'legend.fontsize': 'x-large','figure.figsize': (12, 12),
            'axes.labelsize': 'x-large','axes.titlesize':'x-large',
            'xtick.labelsize':'x-large','ytick.labelsize':'x-large'}
    plt.rcParams.update(params)
    fig = plt.figure()
    matches_t1, matches_t2 = map(list, zip(*matches))
    plt.scatter(matches_t1, matches_t2, marker='x', color='black')
    plt.xlabel('Time in query signal (s)')
    if self_compare:
        plt.ylabel('Time in query signal (s)')
    else:
        plt.ylabel('Time in document signal (s)')
    plt.show()

E:\Program Files\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def get_stft(snd_file, type = 0):
    if type == 0: sound = pydub.AudioSegment.from_file(snd_file, format="wav") 
    if type == 1: sound = pydub.AudioSegment.from_file(snd_file, format="m4a") 
    samples = get_samples(sound)
    duration = sound.duration_seconds
    window_ms = 50.0
    hop_ms = 10.0
    nperseg = (window_ms / 1000.0) * sound.frame_rate
    noverlap = ((window_ms - hop_ms) / 1000.0) * sound.frame_rate
    f, t, Zxx  = signal.stft(samples, fs = sound.frame_rate, window = 'hann', nperseg = nperseg, noverlap = noverlap, nfft = 50*4/1000*sound.frame_rate)
    f /= 1000.0
    dB = 20 * np.log10(np.abs(Zxx / abs(Zxx).max()))
    return t, f, dB

In [3]:
def get_fingerprint(snd_file, type = 0):
    t, f, dB = get_stft(snd_file, type)
    anchors = generate_anchors(t, f, dB)
    Fp = generate_fingerprint(anchors)
    return Fp

In [4]:
import pickle

In [ ]:
### Write a program to locate the files in the kiki-bouba collection from which all these queries come.

### generate query fingerprints and write to query.pckl

f = open('query.pckl', 'wb')
F_q_arr = []
for i in range(1,7):
    query_file = "Q" + str(i) + ".wav"
    F_q = get_fingerprint(query_file)
    print("fingerprint generated: Q" + str(i))
    F_q_arr.append(F_q)
pickle.dump(F_q_arr, f)
f.close()


fingerprint generated: Q1
fingerprint generated: Q2
fingerprint generated: Q3
fingerprint generated: Q4
fingerprint generated: Q5
fingerprint generated: Q6


In [5]:
f = open('query.pckl', 'rb')
F_q_arr_read = pickle.load(f)
f.close()
len(F_q_arr_read)

6

In [ ]:
f = open('kiki1.pckl', 'wb')
F_kiki1_arr = []
for i in range(1,11):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki1_arr.append(F_kiki)
pickle.dump(F_kiki1_arr, f)
f.close()

fingerprint generated: kiki_0001
fingerprint generated: kiki_0002
fingerprint generated: kiki_0003
fingerprint generated: kiki_0004
fingerprint generated: kiki_0005
fingerprint generated: kiki_0006
fingerprint generated: kiki_0007
fingerprint generated: kiki_0008
fingerprint generated: kiki_0009
fingerprint generated: kiki_0010


In [ ]:
f = open('bouba1.pckl', 'wb')
F_bouba1_arr = []
for i in range(1,11):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba1_arr.append(F_bouba)
pickle.dump(F_bouba1_arr, f)
f.close()

fingerprint generated: bouba_0001
fingerprint generated: bouba_0002
fingerprint generated: bouba_0003
fingerprint generated: bouba_0004
fingerprint generated: bouba_0005
fingerprint generated: bouba_0006
fingerprint generated: bouba_0007


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0008
fingerprint generated: bouba_0009


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0010


In [ ]:
f = open('kiki2.pckl', 'wb')
F_kiki2_arr = []
for i in range(11,21):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki2_arr.append(F_kiki)
pickle.dump(F_kiki2_arr, f)
f.close()

fingerprint generated: kiki_0011


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0012
fingerprint generated: kiki_0013
fingerprint generated: kiki_0014
fingerprint generated: kiki_0015
fingerprint generated: kiki_0016
fingerprint generated: kiki_0017
fingerprint generated: kiki_0018
fingerprint generated: kiki_0019
fingerprint generated: kiki_0020


In [ ]:
f = open('bouba2.pckl', 'wb')
F_bouba2_arr = []
for i in range(11,21):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba2_arr.append(F_bouba)
pickle.dump(F_bouba2_arr, f)
f.close()

fingerprint generated: bouba_0011


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0012
fingerprint generated: bouba_0013


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0014


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0015
fingerprint generated: bouba_0016
fingerprint generated: bouba_0017
fingerprint generated: bouba_0018
fingerprint generated: bouba_0019
fingerprint generated: bouba_0020


In [ ]:
f = open('kiki3.pckl', 'wb')
F_kiki3_arr = []
for i in range(21,31):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki3_arr.append(F_kiki)
pickle.dump(F_kiki3_arr, f)
f.close()

fingerprint generated: kiki_0021
fingerprint generated: kiki_0022
fingerprint generated: kiki_0023
fingerprint generated: kiki_0024
fingerprint generated: kiki_0025
fingerprint generated: kiki_0026
fingerprint generated: kiki_0027
fingerprint generated: kiki_0028
fingerprint generated: kiki_0029
fingerprint generated: kiki_0030


In [ ]:
f = open('bouba3.pckl', 'wb')
F_bouba3_arr = []
for i in range(21,31):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba3_arr.append(F_bouba)
pickle.dump(F_bouba3_arr, f)
f.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0021
fingerprint generated: bouba_0022
fingerprint generated: bouba_0023
fingerprint generated: bouba_0024
fingerprint generated: bouba_0025
fingerprint generated: bouba_0026
fingerprint generated: bouba_0027
fingerprint generated: bouba_0028
fingerprint generated: bouba_0029
fingerprint generated: bouba_0030


In [ ]:
f = open('kiki4.pckl', 'wb')
F_kiki4_arr = []
for i in range(31,41):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki4_arr.append(F_kiki)
pickle.dump(F_kiki4_arr, f)
f.close()

fingerprint generated: kiki_0031
fingerprint generated: kiki_0032
fingerprint generated: kiki_0033
fingerprint generated: kiki_0034
fingerprint generated: kiki_0035
fingerprint generated: kiki_0036


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0037


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0038
fingerprint generated: kiki_0039
fingerprint generated: kiki_0040


In [ ]:
f = open('bouba4.pckl', 'wb')
F_bouba4_arr = []
for i in range(31,41):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba4_arr.append(F_bouba)
pickle.dump(F_bouba4_arr, f)
f.close()

fingerprint generated: bouba_0031


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0032
fingerprint generated: bouba_0033
fingerprint generated: bouba_0034
fingerprint generated: bouba_0035
fingerprint generated: bouba_0036
fingerprint generated: bouba_0037
fingerprint generated: bouba_0038
fingerprint generated: bouba_0039
fingerprint generated: bouba_0040


In [ ]:
f = open('kiki5.pckl', 'wb')
F_kiki5_arr = []
for i in range(41,51):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki5_arr.append(F_kiki)
pickle.dump(F_kiki5_arr, f)
f.close()

fingerprint generated: kiki_0041
fingerprint generated: kiki_0042
fingerprint generated: kiki_0043
fingerprint generated: kiki_0044
fingerprint generated: kiki_0045
fingerprint generated: kiki_0046
fingerprint generated: kiki_0047
fingerprint generated: kiki_0048
fingerprint generated: kiki_0049
fingerprint generated: kiki_0050


In [ ]:
f = open('bouba5.pckl', 'wb')
F_bouba5_arr = []
for i in range(41,51):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba5_arr.append(F_bouba)
pickle.dump(F_bouba5_arr, f)
f.close()

fingerprint generated: bouba_0041
fingerprint generated: bouba_0042
fingerprint generated: bouba_0043
fingerprint generated: bouba_0044
fingerprint generated: bouba_0045
fingerprint generated: bouba_0046
fingerprint generated: bouba_0047
fingerprint generated: bouba_0048
fingerprint generated: bouba_0049
fingerprint generated: bouba_0050


In [ ]:
f = open('kiki6.pckl', 'wb')
F_kiki6_arr = []
for i in range(51,61):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki6_arr.append(F_kiki)
pickle.dump(F_kiki6_arr, f)
f.close()

fingerprint generated: kiki_0051
fingerprint generated: kiki_0052
fingerprint generated: kiki_0053
fingerprint generated: kiki_0054
fingerprint generated: kiki_0055
fingerprint generated: kiki_0056
fingerprint generated: kiki_0057
fingerprint generated: kiki_0058
fingerprint generated: kiki_0059
fingerprint generated: kiki_0060


In [ ]:
f = open('bouba6.pckl', 'wb')
F_bouba6_arr = []
for i in range(51,61):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba6_arr.append(F_bouba)
pickle.dump(F_bouba6_arr, f)
f.close()

fingerprint generated: bouba_0051
fingerprint generated: bouba_0052
fingerprint generated: bouba_0053
fingerprint generated: bouba_0054
fingerprint generated: bouba_0055
fingerprint generated: bouba_0056
fingerprint generated: bouba_0057


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0058
fingerprint generated: bouba_0059
fingerprint generated: bouba_0060


In [ ]:
f = open('kiki7.pckl', 'wb')
F_kiki7_arr = []
for i in range(61,71):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki7_arr.append(F_kiki)
pickle.dump(F_kiki7_arr, f)
f.close()

fingerprint generated: kiki_0061
fingerprint generated: kiki_0062
fingerprint generated: kiki_0063
fingerprint generated: kiki_0064
fingerprint generated: kiki_0065


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0066
fingerprint generated: kiki_0067
fingerprint generated: kiki_0068
fingerprint generated: kiki_0069
fingerprint generated: kiki_0070


In [ ]:
f = open('bouba7.pckl', 'wb')
F_bouba7_arr = []
for i in range(61,71):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba7_arr.append(F_bouba)
pickle.dump(F_bouba7_arr, f)
f.close()

fingerprint generated: bouba_0061
fingerprint generated: bouba_0062


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0063
fingerprint generated: bouba_0064
fingerprint generated: bouba_0065
fingerprint generated: bouba_0066
fingerprint generated: bouba_0067
fingerprint generated: bouba_0068
fingerprint generated: bouba_0069
fingerprint generated: bouba_0070


In [ ]:
f = open('kiki8.pckl', 'wb')
F_kiki8_arr = []
for i in range(71,81):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki8_arr.append(F_kiki)
pickle.dump(F_kiki8_arr, f)
f.close()

fingerprint generated: kiki_0071
fingerprint generated: kiki_0072
fingerprint generated: kiki_0073
fingerprint generated: kiki_0074
fingerprint generated: kiki_0075
fingerprint generated: kiki_0076
fingerprint generated: kiki_0077
fingerprint generated: kiki_0078
fingerprint generated: kiki_0079
fingerprint generated: kiki_0080


In [ ]:
f = open('bouba8.pckl', 'wb')
F_bouba8_arr = []
for i in range(71,81):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba8_arr.append(F_bouba)
pickle.dump(F_bouba8_arr, f)
f.close()

fingerprint generated: bouba_0071
fingerprint generated: bouba_0072
fingerprint generated: bouba_0073


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0074


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0075
fingerprint generated: bouba_0076
fingerprint generated: bouba_0077
fingerprint generated: bouba_0078
fingerprint generated: bouba_0079
fingerprint generated: bouba_0080


In [ ]:
f = open('kiki9.pckl', 'wb')
F_kiki9_arr = []
for i in range(81,91):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki9_arr.append(F_kiki)
pickle.dump(F_kiki9_arr, f)
f.close()

fingerprint generated: kiki_0081
fingerprint generated: kiki_0082
fingerprint generated: kiki_0083
fingerprint generated: kiki_0084
fingerprint generated: kiki_0085
fingerprint generated: kiki_0086
fingerprint generated: kiki_0087
fingerprint generated: kiki_0088
fingerprint generated: kiki_0089
fingerprint generated: kiki_0090


In [ ]:
f = open('bouba9.pckl', 'wb')
F_bouba9_arr = []
for i in range(81,91):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba9_arr.append(F_bouba)
pickle.dump(F_bouba9_arr, f)
f.close()

fingerprint generated: bouba_0081
fingerprint generated: bouba_0082
fingerprint generated: bouba_0083
fingerprint generated: bouba_0084
fingerprint generated: bouba_0085
fingerprint generated: bouba_0086
fingerprint generated: bouba_0087
fingerprint generated: bouba_0088


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0089
fingerprint generated: bouba_0090


In [ ]:
f = open('kiki10.pckl', 'wb')
F_kiki10_arr = []
for i in range(91,101):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki10_arr.append(F_kiki)
pickle.dump(F_kiki10_arr, f)
f.close()

fingerprint generated: kiki_0091


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0092
fingerprint generated: kiki_0093
fingerprint generated: kiki_0094
fingerprint generated: kiki_0095
fingerprint generated: kiki_0096
fingerprint generated: kiki_0097
fingerprint generated: kiki_0098
fingerprint generated: kiki_0099
fingerprint generated: kiki_0100


In [ ]:
f = open('bouba10.pckl', 'wb')
F_bouba10_arr = []
for i in range(91,101):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba10_arr.append(F_bouba)
pickle.dump(F_bouba10_arr, f)
f.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0091
fingerprint generated: bouba_0092
fingerprint generated: bouba_0093
fingerprint generated: bouba_0094


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0095
fingerprint generated: bouba_0096
fingerprint generated: bouba_0097


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0098
fingerprint generated: bouba_0099


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0100


In [ ]:
f = open('kiki11.pckl', 'wb')
F_kiki11_arr = []
for i in range(101,111):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki11_arr.append(F_kiki)
pickle.dump(F_kiki11_arr, f)
f.close()

fingerprint generated: kiki_0101
fingerprint generated: kiki_0102


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0103
fingerprint generated: kiki_0104
fingerprint generated: kiki_0105
fingerprint generated: kiki_0106
fingerprint generated: kiki_0107
fingerprint generated: kiki_0108
fingerprint generated: kiki_0109
fingerprint generated: kiki_0110


In [ ]:
f = open('bouba11.pckl', 'wb')
F_bouba11_arr = []
for i in range(101,111):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba11_arr.append(F_bouba)
pickle.dump(F_bouba11_arr, f)
f.close()

fingerprint generated: bouba_0101
fingerprint generated: bouba_0102
fingerprint generated: bouba_0103
fingerprint generated: bouba_0104
fingerprint generated: bouba_0105
fingerprint generated: bouba_0106
fingerprint generated: bouba_0107
fingerprint generated: bouba_0108
fingerprint generated: bouba_0109


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0110


In [ ]:
f = open('kiki12.pckl', 'wb')
F_kiki12_arr = []
for i in range(111,121):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki12_arr.append(F_kiki)
pickle.dump(F_kiki12_arr, f)
f.close()

fingerprint generated: kiki_0111
fingerprint generated: kiki_0112
fingerprint generated: kiki_0113
fingerprint generated: kiki_0114
fingerprint generated: kiki_0115
fingerprint generated: kiki_0116
fingerprint generated: kiki_0117


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0118
fingerprint generated: kiki_0119
fingerprint generated: kiki_0120


In [ ]:
f = open('bouba12.pckl', 'wb')
F_bouba12_arr = []
for i in range(111,121):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba12_arr.append(F_bouba)
pickle.dump(F_bouba12_arr, f)
f.close()

fingerprint generated: bouba_0111


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0112
fingerprint generated: bouba_0113
fingerprint generated: bouba_0114
fingerprint generated: bouba_0115
fingerprint generated: bouba_0116
fingerprint generated: bouba_0117
fingerprint generated: bouba_0118
fingerprint generated: bouba_0119
fingerprint generated: bouba_0120


In [ ]:
f = open('kiki13.pckl', 'wb')
F_kiki13_arr = []
for i in range(121,131):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki13_arr.append(F_kiki)
pickle.dump(F_kiki13_arr, f)
f.close()

fingerprint generated: kiki_0121


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0122
fingerprint generated: kiki_0123
fingerprint generated: kiki_0124
fingerprint generated: kiki_0125
fingerprint generated: kiki_0126
fingerprint generated: kiki_0127


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0128
fingerprint generated: kiki_0129
fingerprint generated: kiki_0130


In [ ]:
f = open('bouba13.pckl', 'wb')
F_bouba13_arr = []
for i in range(121,131):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba13_arr.append(F_bouba)
pickle.dump(F_bouba13_arr, f)
f.close()

fingerprint generated: bouba_0121
fingerprint generated: bouba_0122


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0123


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0124
fingerprint generated: bouba_0125
fingerprint generated: bouba_0126
fingerprint generated: bouba_0127


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0128
fingerprint generated: bouba_0129


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0130


In [ ]:
f = open('kiki14.pckl', 'wb')
F_kiki14_arr = []
for i in range(131,141):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki14_arr.append(F_kiki)
pickle.dump(F_kiki14_arr, f)
f.close()

fingerprint generated: kiki_0131
fingerprint generated: kiki_0132
fingerprint generated: kiki_0133
fingerprint generated: kiki_0134
fingerprint generated: kiki_0135
fingerprint generated: kiki_0136
fingerprint generated: kiki_0137
fingerprint generated: kiki_0138
fingerprint generated: kiki_0139
fingerprint generated: kiki_0140


In [ ]:
f = open('bouba14.pckl', 'wb')
F_bouba14_arr = []
for i in range(131,141):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba14_arr.append(F_bouba)
pickle.dump(F_bouba14_arr, f)
f.close()

fingerprint generated: bouba_0131


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0132
fingerprint generated: bouba_0133
fingerprint generated: bouba_0134
fingerprint generated: bouba_0135
fingerprint generated: bouba_0136


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0137
fingerprint generated: bouba_0138
fingerprint generated: bouba_0139
fingerprint generated: bouba_0140


In [ ]:
f = open('kiki15.pckl', 'wb')
F_kiki15_arr = []
for i in range(141,151):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki15_arr.append(F_kiki)
pickle.dump(F_kiki15_arr, f)
f.close()

fingerprint generated: kiki_0141
fingerprint generated: kiki_0142
fingerprint generated: kiki_0143
fingerprint generated: kiki_0144


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0145
fingerprint generated: kiki_0146
fingerprint generated: kiki_0147
fingerprint generated: kiki_0148
fingerprint generated: kiki_0149
fingerprint generated: kiki_0150


In [ ]:
f = open('bouba15.pckl', 'wb')
F_bouba15_arr = []
for i in range(141,151):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba15_arr.append(F_bouba)
pickle.dump(F_bouba15_arr, f)
f.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0141
fingerprint generated: bouba_0142
fingerprint generated: bouba_0143


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0144
fingerprint generated: bouba_0145
fingerprint generated: bouba_0146


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0147
fingerprint generated: bouba_0148
fingerprint generated: bouba_0149
fingerprint generated: bouba_0150


In [ ]:
f = open('kiki16.pckl', 'wb')
F_kiki16_arr = []
for i in range(151,161):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki16_arr.append(F_kiki)
pickle.dump(F_kiki16_arr, f)
f.close()

fingerprint generated: kiki_0151
fingerprint generated: kiki_0152
fingerprint generated: kiki_0153
fingerprint generated: kiki_0154
fingerprint generated: kiki_0155
fingerprint generated: kiki_0156
fingerprint generated: kiki_0157
fingerprint generated: kiki_0158
fingerprint generated: kiki_0159
fingerprint generated: kiki_0160


In [ ]:
f = open('bouba16.pckl', 'wb')
F_bouba16_arr = []
for i in range(151,161):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba16_arr.append(F_bouba)
pickle.dump(F_bouba16_arr, f)
f.close()

fingerprint generated: bouba_0151


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0152


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0153
fingerprint generated: bouba_0154
fingerprint generated: bouba_0155
fingerprint generated: bouba_0156
fingerprint generated: bouba_0157
fingerprint generated: bouba_0158
fingerprint generated: bouba_0159
fingerprint generated: bouba_0160


In [ ]:
f = open('kiki17.pckl', 'wb')
F_kiki17_arr = []
for i in range(161,171):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki17_arr.append(F_kiki)
pickle.dump(F_kiki17_arr, f)
f.close()

fingerprint generated: kiki_0161
fingerprint generated: kiki_0162
fingerprint generated: kiki_0163
fingerprint generated: kiki_0164
fingerprint generated: kiki_0165


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0166
fingerprint generated: kiki_0167
fingerprint generated: kiki_0168
fingerprint generated: kiki_0169
fingerprint generated: kiki_0170


In [ ]:
f = open('bouba17.pckl', 'wb')
F_bouba17_arr = []
for i in range(161,171):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba17_arr.append(F_bouba)
pickle.dump(F_bouba17_arr, f)
f.close()

fingerprint generated: bouba_0161
fingerprint generated: bouba_0162
fingerprint generated: bouba_0163
fingerprint generated: bouba_0164
fingerprint generated: bouba_0165
fingerprint generated: bouba_0166
fingerprint generated: bouba_0167
fingerprint generated: bouba_0168
fingerprint generated: bouba_0169
fingerprint generated: bouba_0170


In [ ]:
f = open('kiki18.pckl', 'wb')
F_kiki18_arr = []
for i in range(171,181):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki18_arr.append(F_kiki)
pickle.dump(F_kiki18_arr, f)
f.close()

fingerprint generated: kiki_0171
fingerprint generated: kiki_0172
fingerprint generated: kiki_0173
fingerprint generated: kiki_0174
fingerprint generated: kiki_0175
fingerprint generated: kiki_0176
fingerprint generated: kiki_0177
fingerprint generated: kiki_0178
fingerprint generated: kiki_0179
fingerprint generated: kiki_0180


In [ ]:
f = open('bouba18.pckl', 'wb')
F_bouba18_arr = []
for i in range(171,181):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba18_arr.append(F_bouba)
pickle.dump(F_bouba18_arr, f)
f.close()

fingerprint generated: bouba_0171


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0172


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0173
fingerprint generated: bouba_0174
fingerprint generated: bouba_0175
fingerprint generated: bouba_0176
fingerprint generated: bouba_0177


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0178
fingerprint generated: bouba_0179
fingerprint generated: bouba_0180


In [ ]:
f = open('kiki19.pckl', 'wb')
F_kiki19_arr = []
for i in range(181,191):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki19_arr.append(F_kiki)
pickle.dump(F_kiki19_arr, f)
f.close()

fingerprint generated: kiki_0181
fingerprint generated: kiki_0182
fingerprint generated: kiki_0183
fingerprint generated: kiki_0184
fingerprint generated: kiki_0185


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0186
fingerprint generated: kiki_0187
fingerprint generated: kiki_0188
fingerprint generated: kiki_0189
fingerprint generated: kiki_0190


In [ ]:
f = open('bouba19.pckl', 'wb')
F_bouba19_arr = []
for i in range(181,191):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba19_arr.append(F_bouba)
pickle.dump(F_bouba19_arr, f)
f.close()

fingerprint generated: bouba_0181
fingerprint generated: bouba_0182
fingerprint generated: bouba_0183
fingerprint generated: bouba_0184
fingerprint generated: bouba_0185
fingerprint generated: bouba_0186
fingerprint generated: bouba_0187
fingerprint generated: bouba_0188
fingerprint generated: bouba_0189
fingerprint generated: bouba_0190


In [ ]:
f = open('kiki20.pckl', 'wb')
F_kiki20_arr = []
for i in range(191,201):
    index = str(i).zfill(4)
    kiki_file = "kiki_" + index + ".m4a"
    F_kiki = get_fingerprint(kiki_file, 1)
    print("fingerprint generated: kiki_" + index)
    F_kiki20_arr.append(F_kiki)
pickle.dump(F_kiki20_arr, f)
f.close()

fingerprint generated: kiki_0191
fingerprint generated: kiki_0192
fingerprint generated: kiki_0193
fingerprint generated: kiki_0194
fingerprint generated: kiki_0195


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0196
fingerprint generated: kiki_0197
fingerprint generated: kiki_0198


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: kiki_0199
fingerprint generated: kiki_0200


In [ ]:
f = open('bouba20.pckl', 'wb')
F_bouba20_arr = []
for i in range(191,201):
    index = str(i).zfill(4)
    bouba_file = "bouba_" + index + ".m4a"
    F_bouba = get_fingerprint(bouba_file, 1)
    print("fingerprint generated: bouba_" + index)
    F_bouba20_arr.append(F_bouba)
pickle.dump(F_bouba20_arr, f)
f.close()

fingerprint generated: bouba_0191
fingerprint generated: bouba_0192


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log10
  if sys.path[0] == '':


fingerprint generated: bouba_0193
fingerprint generated: bouba_0194
fingerprint generated: bouba_0195
fingerprint generated: bouba_0196
fingerprint generated: bouba_0197
fingerprint generated: bouba_0198
fingerprint generated: bouba_0199
fingerprint generated: bouba_0200


In [ ]:
!zip -r 'NewFileName.zip' 'kiki20.pckl'

  adding: kiki20.pckl (deflated 74%)


In [35]:
def generate_num_of_matches_new(F1, F2): # the old one was soooo slow 
    F1_hashes = np.array(F1)[:,1]
    F2_hashes = np.array(F2)[:,1]
    n = np.intersect1d(F1_hashes, F2_hashes).shape[0]
    return n

In [34]:
F1_hashes = np.array(F_q1)[:,1]
F2_hashes = np.array(F_kiki_arr_read[0])[:,1]
np.intersect1d(F1_hashes, F2_hashes).shape[0]

<ipython-input-34-94c713aa5e31>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F_q1)[:,1]
<ipython-input-34-94c713aa5e31>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F_kiki_arr_read[0])[:,1]


52

In [38]:
F_q1 = F_q_arr_read[0]

max_num_of_matches = np.zeros(20)
max_index = np.zeros(20)
bouba_kiki = np.zeros(20) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(20):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q1)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q1)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-35-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-35-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [39]:
max_num_of_matches

array([345., 297., 362., 313., 349., 290., 335., 317., 304., 301., 347.,
       295., 327., 665., 391., 328., 388., 321., 323., 288.])

In [40]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

bouba0131
665.0


In [41]:
F_q2 = F_q_arr_read[1]

max_num_of_matches = np.zeros(20)
max_index = np.zeros(20)
bouba_kiki = np.zeros(20) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(20):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q2)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q2)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-35-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-35-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [42]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

bouba0051
508.0


In [43]:
F_q3 = F_q_arr_read[2]

max_num_of_matches = np.zeros(20)
max_index = np.zeros(20)
bouba_kiki = np.zeros(20) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(20):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q3)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q3)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-35-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-35-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [44]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

kiki0199
579.0


In [45]:
F_q4 = F_q_arr_read[3]

max_num_of_matches = np.zeros(20)
max_index = np.zeros(20)
bouba_kiki = np.zeros(20) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(20):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q4)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q4)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-35-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-35-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [46]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

kiki0043
368.0


In [47]:
F_q5 = F_q_arr_read[4]

max_num_of_matches = np.zeros(20)
max_index = np.zeros(20)
bouba_kiki = np.zeros(20) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(20):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q5)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q5)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-35-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-35-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [48]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

bouba0051
389.0


In [49]:
F_q6 = F_q_arr_read[5]

max_num_of_matches = np.zeros(20)
max_index = np.zeros(20)
bouba_kiki = np.zeros(20) # 0 = undefined, 1 = kiki, 2 = bouba
for i in range(20):
    index = str(i + 1)
    print(index)
    kiki_name = 'kiki' + index + '.pckl'
    f = open(kiki_name, 'rb')
    F_kiki_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_kiki_tmp = F_kiki_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_kiki_tmp, F_q6)
        print("kiki", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 1
    bouba_name = 'bouba' + index + '.pckl'
    f = open(bouba_name, 'rb')
    F_bouba_arr_read = pickle.load(f)
    f.close()
    for j in range(10):
        F_bouba_tmp = F_bouba_arr_read[j]
        num_of_matches = generate_num_of_matches_new(F_bouba_tmp, F_q6)
        print("bouba", j)
        if num_of_matches > max_num_of_matches[i]:
            max_num_of_matches[i] = num_of_matches
            max_index[i] = j
            bouba_kiki[i] = 2

1


<ipython-input-35-a7d86bc9395b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F1_hashes = np.array(F1)[:,1]
<ipython-input-35-a7d86bc9395b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  F2_hashes = np.array(F2)[:,1]


kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
2
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
3
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
4
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
5
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
6
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bouba 2
bouba 3
bouba 4
bouba 5
bouba 6
bouba 7
bouba 8
bouba 9
7
kiki 0
kiki 1
kiki 2
kiki 3
kiki 4
kiki 5
kiki 6
kiki 7
kiki 8
kiki 9
bouba 0
bouba 1
bo

In [50]:
argmax_matches = np.argmax(max_num_of_matches)
result_index = 10 * argmax_matches + max_index[argmax_matches] + 1
if(bouba_kiki[argmax_matches] == 1): print("kiki" + str(int(result_index)).zfill(4))
if(bouba_kiki[argmax_matches] == 2): print("bouba" + str(int(result_index)).zfill(4))
print(max_num_of_matches[argmax_matches])

kiki0038
109.0
